In [59]:
"""Vehicle Routing Problem"""
from __future__ import print_function
from six.moves import xrange
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2
from sklearn.metrics import pairwise_distances
import numpy as np
import pandas as pd
###########################
# Problem Data Definition #
###########################
class CityBlock():
    """City block definition"""
    @property
    def width(self):
        """Gets Block size West to East"""
        return 228/2

    @property
    def height(self):
        """Gets Block size North to South"""
        return 80

def generate_random_locations(nVisits=200):
    return pd.DataFrame( {'lat': np.random.rand(nVisits)*10, 'lng': np.random.rand(nVisits)*10, 'id': range(nVisits) } )
    
class DataProblem():
    """Stores the data for the problem"""
    def __init__(self):
        """Initializes the data for the problem"""
        self._num_vehicles = 10

        # Locations in block unit
        locations = generate_random_locations(40)
        locations = locations[['lat','lng']].as_matrix()
        self._locations = locations
        # locations in meters using the city block dimension
#         city_block = CityBlock()
#         self._locations = [(
#             loc[0]*city_block.width,
#             loc[1]*city_block.height) for loc in locations]

        self._depot = 0

    @property
    def num_vehicles(self):
        """Gets number of vehicles"""
        return self._num_vehicles

    @property
    def locations(self):
        """Gets locations"""
        return self._locations

    @property
    def num_locations(self):
        """Gets number of locations"""
        return len(self.locations)

    @property
    def depot(self):
        """Gets depot location index"""
        return self._depot

#######################
# Problem Constraints #
#######################
def manhattan_distance(position_1, position_2):
    """Computes the Manhattan distance between two points"""
    return (abs(position_1[0] - position_2[0]) +
            abs(position_1[1] - position_2[1]))

class CreateDistanceEvaluator(object): # pylint: disable=too-few-public-methods
    """Creates callback to return distance between points."""
    def __init__(self, data):
        """Initializes the distance matrix."""
        self._distances = pairwise_distances(data.locations)

#         # precompute distance between location to have distance callback in O(1)
#         for from_node in xrange(data.num_locations):
#             self._distances[from_node] = {}
#             for to_node in xrange(data.num_locations):
#                 if from_node == to_node:
#                     self._distances[from_node][to_node] = 0
#                 else:
#                     self._distances[from_node][to_node] = (
#                         manhattan_distance(
#                             data.locations[from_node],
#                             data.locations[to_node]))

    def distance_evaluator(self, from_node, to_node):
        """Returns the manhattan distance between the two nodes"""
        return self._distances[from_node][to_node]

def add_distance_dimension(routing, distance_evaluator):
    """Add Global Span constraint"""
    distance = "Distance"
    maximum_distance = 3000
    routing.AddDimension(
        distance_evaluator,
        0, # null slack
        maximum_distance, # maximum distance per vehicle
        True, # start cumul to zero
        distance)
    distance_dimension = routing.GetDimensionOrDie(distance)
    # Try to minimize the max distance among vehicles.
    # /!\ It doesn't mean the standard deviation is minimized
    distance_dimension.SetGlobalSpanCostCoefficient(100)

###########
# Printer #
###########
class ConsolePrinter():
    """Print solution to console"""
    def __init__(self, data, routing, assignment):
        """Initializes the printer"""
        self._data = data
        self._routing = routing
        self._assignment = assignment

    @property
    def data(self):
        """Gets problem data"""
        return self._data

    @property
    def routing(self):
        """Gets routing model"""
        return self._routing

    @property
    def assignment(self):
        """Gets routing model"""
        return self._assignment

    def print(self):
        """Prints assignment on console"""
        # Inspect solution.
        total_dist = 0
        for vehicle_id in xrange(self.data.num_vehicles):
            index = self.routing.Start(vehicle_id)
            plan_output = 'Route for vehicle {0}:\n'.format(vehicle_id)
            route_dist = 0
            while not self.routing.IsEnd(index):
                node_index = self.routing.IndexToNode(index)
                next_node_index = self.routing.IndexToNode(
                    self.assignment.Value(self.routing.NextVar(index)))
                route_dist += manhattan_distance(
                    self.data.locations[node_index],
                    self.data.locations[next_node_index])
                plan_output += ' {node_index} -> '.format(
                    node_index=node_index)
                index = self.assignment.Value(self.routing.NextVar(index))

            node_index = self.routing.IndexToNode(index)
            total_dist += route_dist
            plan_output += ' {node_index}\n'.format(
                node_index=node_index)
            plan_output += 'Distance of the route {0}: {dist}\n'.format(
                vehicle_id,
                dist=route_dist)
            print(plan_output)
        print('Total Distance of all routes: {dist}'.format(dist=total_dist))

########
# Main #
########
def test():
    """Entry point of the program"""
    # Instantiate the data problem.
    data = DataProblem()

    # Create Routing Model
    print("no locations: ",data.num_locations, " num vehicles ", data.num_vehicles, ' depot ',data.depot)
    routing = pywrapcp.RoutingModel(data.num_locations, data.num_vehicles, data.depot)
    # Define weight of each edge
    d = CreateDistanceEvaluator(data)
    print('distances shape ',d._distances.shape)
    distance_evaluator = d.distance_evaluator
    routing.SetArcCostEvaluatorOfAllVehicles(distance_evaluator)
    add_distance_dimension(routing, distance_evaluator)

    # Setting first solution heuristic (cheapest addition).
    search_parameters = pywrapcp.RoutingModel.DefaultSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    print(search_parameters)
    # Solve the problem.
    assignment = routing.SolveWithParameters(search_parameters)
    print(assignment)
    printer = ConsolePrinter(data, routing, assignment)
    printer.print()



In [60]:
test()

no locations:  40  num vehicles  10  depot  0
distances shape  (40, 40)
first_solution_strategy: PATH_CHEAPEST_ARC
use_filtered_first_solution_strategy: true
local_search_operators {
  use_relocate: true
  use_relocate_pair: true
  use_exchange: true
  use_cross: true
  use_two_opt: true
  use_or_opt: true
  use_lin_kernighan: true
  use_make_active: true
  use_make_inactive: true
  use_swap_active: true
  use_node_pair_swap_active: true
}
guided_local_search_lambda_coefficient: 0.1
optimization_step: 1
solution_limit: 9223372036854775807
time_limit_ms: 9223372036854775807
lns_time_limit_ms: 100
use_light_propagation: true
fingerprint_arc_cost_evaluators: true

Assignment(Distance0 (0) | Distance1 (6) | Distance2 (8) | Distance3 (7) | Distance4 (1) | Distance5 (11) | Distance6 (2) | Distance7 (8) | Distance8 (6) | Distance9 (1) | Distance10 (3) | Distance11 (6) | Distance12 (2) | Distance13 (2) | Distance14 (1) | Distance15 (1) | Distance16 (3) | Distance17 (8) | Distance18 (3) | Dista

In [50]:
assignment

NameError: name 'assignment' is not defined

In [54]:
locations =  [(4, 4), # depot
                 (2, 0), (8, 0), # row 0
                 (0, 1), (1, 1),
                 (5, 2), (7, 2),
                 (3, 3), (6, 3),
                 (5, 5), (8, 5),
                 (1, 6), (2, 6),
                 (3, 7), (6, 7),
                 (0, 8), (7, 8)]

In [55]:
def make_distance_eval(distances):
    def dist_eval(start,end):
        return distances[start][end]
    return make_distance_eval
        
def VRP(locations,no_vehicles, depot):
    distances  = pairwise_distances(np.array(locations))
    dist_eval = make_distance_eval(distances)
    routing = pywrapcp.RoutingModel(len(locations), no_vehicles, depot)
    routing.SetArcCostEvaluatorOfAllVehicles(dist_eval)
    distance = "Distance"
    maximum_distance = 3000
    routing.AddDimension(
        dist_eval,
        0, # null slack
        maximum_distance, # maximum distance per vehicle
        True, # start cumul to zero
        distance)
    distance_dimension = routing.GetDimensionOrDie(distance)
    # Try to minimize the max distance among vehicles.
    # /!\ It doesn't mean the standard deviation is minimized
    distance_dimension.SetGlobalSpanCostCoefficient(100)
    search_parameters = pywrapcp.RoutingModel.DefaultSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    assignment = routing.SolveWithParameters(search_parameters)
    print(assignment)
#     return assignment

test = VRP(locations, 2 ,6)

SystemError: <built-in function RoutingModel_SolveWithParameters> returned a result with an error set

In [62]:
pywrapcp.RoutingModel??

Init signature: pywrapcp.RoutingModel(*args)
Source:        
class RoutingModel(_object):
    __swig_setmethods__ = {}
    __setattr__ = lambda self, name, value: _swig_setattr(self, RoutingModel, name, value)
    __swig_getmethods__ = {}
    __getattr__ = lambda self, name: _swig_getattr(self, RoutingModel, name)
    __repr__ = _swig_repr
    ROUTING_NOT_SOLVED = _pywrapcp.RoutingModel_ROUTING_NOT_SOLVED
    ROUTING_SUCCESS = _pywrapcp.RoutingModel_ROUTING_SUCCESS
    ROUTING_FAIL = _pywrapcp.RoutingModel_ROUTING_FAIL
    ROUTING_FAIL_TIMEOUT = _pywrapcp.RoutingModel_ROUTING_FAIL_TIMEOUT
    ROUTING_INVALID = _pywrapcp.RoutingModel_ROUTING_INVALID
    __swig_destroy__ = _pywrapcp.delete_RoutingModel
    __del__ = lambda self: None
    if _newclass:
        DefaultModelParameters = staticmethod(_pywrapcp.RoutingModel_DefaultModelParameters)
    else:
        DefaultModelParameters = _pywrapcp.RoutingModel_DefaultModelParameters
    if _newclass:
        DefaultSearchParameters = static